# Hartree-Fock for helium using Legendre polynomials in Python

* [Hartree-Fock](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method)
* [Fermi-Dirac statistics](https://en.wikipedia.org/wiki/Fermi%E2%80%93Dirac_statistics)

Hartree-Fock (HF) is a foundational approximation to the many-body wave function in quantum chemistry that starts with three key ideas
* the wave function for a many-body system comprising non-interacting particles is separable (i.e., is a product of 1-particle wave functions, also called orbitals),
* the wave function for a many-electron system must be antisymmetric with respect to exchange of the coordinates (both space and spin) of any pair of electrons since they are fermions, and
* the variational principle—minimizing the energy by variation of the orbitals.

From HF emerges a clear an easily interpreted understanding of
* the periodic table of the elements (for heavy elements one also needs to include the effects of special relativity), and
* the chemical bond.

Simply stated, the HF wave function is the lowest energy antisymmetric product of N orbitals.  It is a mean-field model—i.e., each electron experiences only the average potential due to the other electrons.

I won't derive the general HF energy equation since it requires introducing spin and at this stage is not very instructive.  Let's immediately jump to the equations for the helium atom (2 electrons and point nucleus of charge +2).  Let the orbital (function in 3D space) again be denoted as $\psi(r)$.  The energy functional (i.e., a function of a function) is
$$E[\psi] = 2 \left \langle \psi \middle| \hat{h} \middle| \psi \right \rangle + \left(\psi \psi | \psi \psi \right)$$
where, $\hat{h}$ is the one-electron Hamiltonion operator that we've seen before
$$\hat{h} = -\frac{1}{2} \nabla^2 - \frac{2}{r}$$
that includes the kinetic energy and nuclear attraction terms.  The new piece is the electron repulsion integral
$$\left(\psi \psi | \psi \psi \right) = \int \int dr_1^3 dr_2^3 \psi(r_1)^2 \frac{1}{|r_1 - r_2|} \psi(r_2)^2 $$
which is the electrostatic repulsion energy between the two electrons.  Note this is a two-electron integral that therefore involves integrating over 6 dimensions!  In general molecules, calculating these integrals is a major computational task. However, the spherical symmetry of atoms reduces this to just 2 radial dimensions.  

"Differentiating" the energy with respect to the wave function yields the gradient
$$\frac{\delta E}{\delta \psi} = 4 \left(\hat{h} \psi(r) + u(r) \psi(r) \right),$$
where 
$$ u(r) = \int ds^3 \frac{\rho(s)}{|r-s|}$$
which we recognize as the electrostatic potential due the charge density $\rho(s) = \psi(s)^2$. With this definition of $u(r)$, the two-electron integral in the energy becomes
$$\left(\psi \psi | \psi \psi \right) = \int dr^3 u(r) \rho(r).$$

Let's now use the spherical symmetry of the ground state helium atom to simplify things.  Above $r$, $s$, etc., were 3D vectors --- now they will just be scalar values representing the distance from the origin.
$$\hat{h}(r) \psi(r) = -\frac{1}{2} \frac{1}{r} \! \frac{d^2}{dr^2} r \psi(r) - \frac{2}{r} \psi(r)$$
and
$$u(r) = 4 \pi \left( \frac{1}{r} \int_0^r ds s^2 \rho(s)  +  \int_r^\infty ds s \rho(s) \right) $$

To test your calculation of $u(r)$ you can use the potential due to a Gaussian with exponent $a$ 
$$g(r) = e^{-a r^2}$$
(test with $a=1$ for simplicity) for which the potential is 
$$\left( \frac{\pi}{a} \right)^{3/2} \frac{\mathrm{erf}\left(\sqrt{a} r \right)}{r}$$

We will again define $\phi(r) = r \psi(r)$ and hence the gradient becomes
$$\frac{\delta E}{\delta \phi} = 4 \left(-\frac{1}{2} \frac{d^2 \phi}{dr^2} - \frac{2}{r} \phi(r) + u(r) \phi(r) \right).$$

Compared to the hydrogen atom, the new pieces are
* $u(r)$ in the gradient
* $\left(\psi \psi | \psi \psi \right)$ in the energy

We are solving in a finite box, replacing $\infty$ with $L$ and we are usually integrating over the range $[0,L]$ that we map to $[-1,1]$ by a change of variables so that we can use the standard Gauss-Legendre quadrature.  To calculate $u(r)$, we just need to modify the range of integration—the first integral is over $[0,r]$ and the second is over $[r,L]$.  *But*, note that $r$ is now a variable so for each value of $r$ at which you want to compute the potential you need to compute the integrals that define $u$.  So just make a loop over the values of $r$ that you need, and insert the integral code inside that loop (being sure to do the change of variables to respect the required ranges of integration).  

Very likely, you will be wanting to compute the electrostatic potential matrix over the basis functions ($b_i(r)$).  *Note*, that since density depends on the orbital that this matrix *changes* every time you update the wave function and therefore must be recomputed every time.
$$
u_{ij} = \left\langle b_i \middle| u \middle| b_j \right\rangle  \\
  =  4 \pi \int_0^L dr r^2 b_i(r) u(r) b_j(r)
 $$
You will compute this integral using Gaussian quadrature.  At each quadrature point you will evaluate the potential by computing the integrals as described above.

I suggest that in the Python function you write to calculate the gradient (and also the one for the energy) that you just recompute this matrix every time just to be sure you are using the correct potential.  It will be slow, but correct.  If it is too slow, then once the code is working for a small basis (number of GL polynomials) you can attempt to compute the matrix once and reuse it.
